In [1]:
import datetime
from typing import List, Dict, Optional
import json
import random
from abc import ABC, abstractmethod
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

# Constants
DEFAULT_REMINDER_DAYS = 3
MIN_DAYS_FOR_AUTOPAY_SUGGESTION = 3

In [2]:
class Payment:
    def __init__(self, name: str, amount: float, due_date: datetime.date, 
                 payment_type: str, is_recurring: bool = True, 
                 recurrence_period: Optional[str] = None):
        self.name = name
        self.amount = amount
        self.due_date = due_date
        self.payment_type = payment_type  # 'bill', 'emi', 'subscription'
        self.is_recurring = is_recurring
        self.recurrence_period = recurrence_period  # 'monthly', 'yearly', 'weekly'
        
    def to_dict(self):
        return {
            'name': self.name,
            'amount': self.amount,
            'due_date': self.due_date.isoformat(),
            'payment_type': self.payment_type,
            'is_recurring': self.is_recurring,
            'recurrence_period': self.recurrence_period
        }
    
    @classmethod
    def from_dict(cls, data: Dict):
        return cls(
            name=data['name'],
            amount=data['amount'],
            due_date=datetime.date.fromisoformat(data['due_date']),
            payment_type=data['payment_type'],
            is_recurring=data['is_recurring'],
            recurrence_period=data.get('recurrence_period')
        )

In [3]:
class PaymentHistory:
    def __init__(self):
        self.payments = []
        self.payment_history = []  # Track actual payments made
    
    def add_payment(self, payment: Payment):
        self.payments.append(payment)
    
    def record_payment_made(self, payment: Payment, payment_date: datetime.date):
        self.payment_history.append({
            'payment': payment.to_dict(),
            'payment_date': payment_date.isoformat()
        })
    
    def get_upcoming_payments(self, days_ahead: int = 30) -> List[Payment]:
        today = datetime.date.today()
        end_date = today + datetime.timedelta(days=days_ahead)
        
        upcoming = []
        for payment in self.payments:
            if payment.due_date >= today and payment.due_date <= end_date:
                upcoming.append(payment)
        
        return sorted(upcoming, key=lambda x: x.due_date)
    
    def get_payment_pattern(self, payment_name: str) -> Optional[Dict]:
        """Analyze payment pattern for a specific bill/emi"""
        relevant_payments = [
            p for p in self.payment_history 
            if p['payment']['name'] == payment_name
        ]
        
        if not relevant_payments:
            return None
        
        # Calculate average days paid before/after due date
        diffs = []
        amounts = []
        
        for record in relevant_payments:
            due_date = datetime.date.fromisoformat(record['payment']['due_date'])
            payment_date = datetime.date.fromisoformat(record['payment_date'])
            diff = (due_date - payment_date).days
            diffs.append(diff)
            amounts.append(record['payment']['amount'])
        
        return {
            'name': payment_name,
            'count': len(relevant_payments),
            'avg_days_before_due': np.mean(diffs),
            'amount_consistency': np.std(amounts) / np.mean(amounts) if len(amounts) > 1 else 0,
            'last_payment_date': max(
                datetime.date.fromisoformat(r['payment_date']) for r in relevant_payments
            )
        }
    
    def save_to_file(self, filename: str):
        data = {
            'payments': [p.to_dict() for p in self.payments],
            'payment_history': self.payment_history
        }
        with open(filename, 'w') as f:
            json.dump(data, f)
    
    @classmethod
    def load_from_file(cls, filename: str):
        with open(filename, 'r') as f:
            data = json.load(f)
        
        history = cls()
        history.payments = [Payment.from_dict(p) for p in data['payments']]
        history.payment_history = data['payment_history']
        return history

In [4]:
class NotificationSystem(ABC):
    @abstractmethod
    def send_notification(self, message: str, payment: Payment):
        pass

class ConsoleNotification(NotificationSystem):
    def send_notification(self, message: str, payment: Payment):
        print(f"NOTIFICATION: {message}")
        print(f"Payment: {payment.name} - ${payment.amount} due on {payment.due_date}")

In [5]:
class PaymentPredictor:
    def __init__(self, payment_history: PaymentHistory):
        self.payment_history = payment_history
    
    def predict_upcoming_payments(self, days_ahead: int = 30) -> List[Payment]:
        """Predict upcoming payments based on history"""
        upcoming = self.payment_history.get_upcoming_payments(days_ahead)
        predictions = []
        
        for payment in upcoming:
            if payment.is_recurring and payment.recurrence_period:
                predictions.extend(self._predict_future_recurring(payment, days_ahead))
            else:
                predictions.append(payment)
        
        return sorted(predictions, key=lambda x: x.due_date)
    
    def _predict_future_recurring(self, payment: Payment, days_ahead: int) -> List[Payment]:
        """Predict future instances of recurring payments"""
        today = datetime.date.today()
        end_date = today + datetime.timedelta(days=days_ahead)
        predictions = []
        
        current_date = payment.due_date
        while current_date <= end_date:
            if current_date >= today:
                predictions.append(Payment(
                    name=payment.name,
                    amount=payment.amount,
                    due_date=current_date,
                    payment_type=payment.payment_type,
                    is_recurring=True,
                    recurrence_period=payment.recurrence_period
                ))
            
            # Calculate next due date
            if payment.recurrence_period == 'monthly':
                current_date += relativedelta(months=1)
            elif payment.recurrence_period == 'yearly':
                current_date += relativedelta(years=1)
            elif payment.recurrence_period == 'weekly':
                current_date += datetime.timedelta(weeks=1)
            else:
                break
        
        return predictions
    
    def should_suggest_autopay(self, payment_name: str) -> bool:
        """Determine if autopay should be suggested"""
        pattern = self.payment_history.get_payment_pattern(payment_name)
        if not pattern:
            return False
        
        return (pattern['count'] >= MIN_DAYS_FOR_AUTOPAY_SUGGESTION and 
                pattern['amount_consistency'] < 0.2 and 
                pattern['avg_days_before_due'] >= 0)

In [6]:
class PaymentReminder:
    def __init__(self, payment_history: PaymentHistory, 
                 notification_system: NotificationSystem,
                 reminder_days: int = DEFAULT_REMINDER_DAYS):
        self.payment_history = payment_history
        self.notification_system = notification_system
        self.reminder_days = reminder_days
        self.predictor = PaymentPredictor(payment_history)
    
    def check_and_remind(self):
        """Check for upcoming payments and send reminders"""
        today = datetime.date.today()
        upcoming = self.predictor.predict_upcoming_payments(self.reminder_days + 7)
        
        for payment in upcoming:
            days_until_due = (payment.due_date - today).days
            
            if 0 <= days_until_due <= self.reminder_days:
                self.notification_system.send_notification(
                    f"Upcoming payment due in {days_until_due} days", 
                    payment
                )
                
                if self.predictor.should_suggest_autopay(payment.name):
                    self.notification_system.send_notification(
                        f"Consider auto-pay for {payment.name} (consistent on-time payments)", 
                        payment
                    )
            elif days_until_due < 0:
                self.notification_system.send_notification(
                    f"OVERDUE by {-days_until_due} days!", 
                    payment
                )

In [7]:
def generate_sample_data(payment_history: PaymentHistory):
    """Generate sample payment data for testing"""
    today = datetime.date.today()
    
    # Recurring bills
    payment_history.add_payment(Payment(
        name="Electric Bill",
        amount=round(random.uniform(80, 120), 2),
        due_date=today + datetime.timedelta(days=5),
        payment_type="bill",
        is_recurring=True,
        recurrence_period="monthly"
    ))
    
    payment_history.add_payment(Payment(
        name="Netflix",
        amount=15.99,
        due_date=today + datetime.timedelta(days=12),
        payment_type="subscription",
        is_recurring=True,
        recurrence_period="monthly"
    ))
    
    # Loan EMI
    payment_history.add_payment(Payment(
        name="Car Loan",
        amount=320.50,
        due_date=today + datetime.timedelta(days=15),
        payment_type="emi",
        is_recurring=True,
        recurrence_period="monthly"
    ))
    
    # Past payments (to establish patterns)
    for i in range(1, 4):
        payment_history.record_payment_made(
            Payment(
                name="Electric Bill",
                amount=round(random.uniform(80, 120), 2),
                due_date=today - datetime.timedelta(days=30*i + 5),
                payment_type="bill"
            ),
            payment_date=today - datetime.timedelta(days=30*i)  # Paid 5 days early
        )

In [9]:
# Initialize with clear output
print("=== Initializing Payment System ===")
payment_history = PaymentHistory()
generate_sample_data(payment_history)

# Print all payments to verify data
print("\n=== All Payments ===")
for payment in payment_history.payments:
    print(f"{payment.name}: ${payment.amount} due {payment.due_date}")

# Set up notifications
print("\n=== Setting Up Reminder System ===")
notifier = ConsoleNotification()
reminder = PaymentReminder(payment_history, notifier, reminder_days=7)  # Increased reminder window

# Force some payments to be due soon
today = datetime.date.today()
payment_history.payments[0].due_date = today + datetime.timedelta(days=1)  # Make first payment due tomorrow
payment_history.payments[1].due_date = today - datetime.timedelta(days=1)  # Make second payment overdue

print("\n=== Checking Reminders ===")
reminder.check_and_remind()

# Show payment patterns
print("\n=== Payment Patterns ===")
for payment in payment_history.payments:
    pattern = payment_history.get_payment_pattern(payment.name)
    if pattern:
        print(f"{payment.name}: Paid {pattern['count']} times (avg {pattern['avg_days_before_due']} days early)")

=== Initializing Payment System ===

=== All Payments ===
Electric Bill: $106.19 due 2025-04-08
Netflix: $15.99 due 2025-04-15
Car Loan: $320.5 due 2025-04-18

=== Setting Up Reminder System ===

=== Checking Reminders ===
NOTIFICATION: Upcoming payment due in 1 days
Payment: Electric Bill - $106.19 due on 2025-04-04

=== Payment Patterns ===
Electric Bill: Paid 3 times (avg -5.0 days early)
